In [1]:
import sys
#sys.path.insert(0,'/eos/home-a/antoniov/SWAN_projects/env/uproot4/lib/python3.7/site-packages')
sys.path.insert(0,'/eos/home-a/antoniov/SWAN_projects/env/uproot/lib/python3.8/site-packages')
print ( sys.path )

import os
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
#import mplhep
from matplotlib.colors import LogNorm
import joblib
from joblib import dump, load

print ( "numpy: {}".format(np.__version__) )
print ( "joblib: {}".format(joblib.__version__) )

import sklearn
import tensorflow as tf
from tensorflow import keras

print ( "sklearn: {}".format(sklearn.__version__) )
print ( "tensorflow: {}".format(tf.__version__) )

['/eos/home-a/antoniov/SWAN_projects/env/uproot/lib/python3.8/site-packages', '/cvmfs/sft.cern.ch/lcg/views/LCG_99/x86_64-centos7-gcc8-opt/python', '/cvmfs/sft.cern.ch/lcg/views/LCG_99/x86_64-centos7-gcc8-opt/lib', '', '/cvmfs/sft.cern.ch/lcg/views/LCG_99/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.8.6-3199b/x86_64-centos7-gcc8-opt/lib/python38.zip', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.8.6-3199b/x86_64-centos7-gcc8-opt/lib/python3.8', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.8.6-3199b/x86_64-centos7-gcc8-opt/lib/python3.8/lib-dynload', '/eos/home-a/antoniov/SWAN_projects/env/uproot/lib/python3.8/site-packages', '/cvmfs/sft.cern.ch/lcg/views/LCG_99/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/IPython/extensions', '/scratch/antoniov/.ipython']
numpy: 1.18.2
joblib: 0.14.0
sklearn: 0.23.2
tensorflow: 2.3.1


In [2]:
#tf.config.experimental.list_physical_devices('GPU')
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
if gpus:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    
print ( gpus )

[]


In [3]:
# get_data, process_data, fiducial_cuts, fiducial_cuts_all, aperture_parametrisation, check_aperture
from processing import *

#proton_selection = "SingleRP"
proton_selection = "MultiRP"

#data_periods = [ "2017B", "2017C1", "2017C2", "2017D", "2017E", "2017F1", "2017F2", "2017F3" ]
data_periods = [ "2017B", "2017C1", "2017E", "2017F1" ]

run_tables = False

run_random_experiments = False

train_model = False
save_model = False
learning_rate_scan = False
run_grid_search = False

scaler_path = ""
model_path = ""
if not train_model:
    scaler_path = "model/standard_scaler_test-multiRP_2021_03_21-21_09_11.joblib"
    model_path = "model/keras_model_test-multiRP_2021_03_21-23_42_07.h5"

### Signal

In [5]:
label = "Elastic"

fileNames_signal = [
    'output/output-MC2017-Elastic-Non3+3-PreSel.h5',
    #'output/output-MC2017-SingleDissociation-Non3+3-PreSel.h5'
]

import time
print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
time_s_ = time.time()

df_counts_signal, df_signal = 2 * [None]

if run_tables:
    with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), complevel=5 ) as store_:

        df_counts_signal_, df_signal_ = get_data( fileNames_signal )
        df_signal_ = process_data( df_signal_, proton_selection, min_mass = 110. )
        
        store_[ "counts" ] = df_counts_signal_
        store_[ "df"] = df_signal_

with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    df_counts_signal = store_[ "counts" ]
    df_signal = store_[ "df"]
        
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

2021/04/15 00:51:41
Total time elapsed: 0


In [6]:
df_counts_signal

All        178402
Muon       166756
Vertex     149608
Protons     91429
dtype: int64

In [7]:
df_signal

,Run,LumiSection,EventNum,Slice,CrossingAngle,MultiRP,Arm,RPId1,RPId2,TrackX1,...,InvMass,ExtraPfCands,Acopl,XiMuMuPlus,XiMuMuMinus,XiMuMu,xlow,xhigh,ylow,yhigh
3,1,1,3,0,150.0,1,1,103,123,7.630859,...,251.395355,16,0.000880,0.004588,0.081510,0.081510,2.422,24.620,-9.698,4.698
6,1,1,12,0,150.0,1,0,3,23,5.149414,...,126.690613,4,0.001160,0.055618,0.001708,0.055618,1.995,24.334,-10.098,4.298
16,1,1,17,0,150.0,1,1,103,123,3.828247,...,263.130524,1,0.000028,0.008177,0.050104,0.050104,2.422,24.620,-9.698,4.698
19,1,1,30,0,150.0,1,0,3,23,5.139893,...,176.417725,1,0.001032,0.046055,0.003999,0.046055,1.995,24.334,-10.098,4.298
24,1,1,13,0,150.0,1,1,103,123,2.742798,...,233.975662,17,0.000317,0.007711,0.042010,0.042010,2.422,24.620,-9.698,4.698
27,1,1,31,0,150.0,1,1,103,123,5.258789,...,266.681091,0,0.001807,0.006805,0.061971,0.061971,2.422,24.620,-9.698,4.698
30,1,1,37,0,150.0,1,0,3,23,4.975342,...,176.964508,0,0.000757,0.054292,0.003414,0.054292,1.995,24.334,-10.098,4.298
42,1,1,52,0,150.0,1,0,3,23,7.566895,...,1466.510620,2,0.000317,0.072588,0.175365,0.072588,1.995,24.334,-10.098,4.298
43,1,1,52,0,150.0,1,1,103,123,11.388184,...,1466.510620,2,0.000317,0.072588,0.175365,0.175365,2.422,24.620,-9.698,4.698
46,1,1,57,0,150.0,1,1,103,123,3.813232,...,115.234695,0,0.000651,0.001538,0.051120,0.051120,2.422,24.620,-9.698,4.698


In [8]:
L_B  = 2.360904801;
L_C1 = 5.313012839;
L_E  = 8.958810514;
L_F1 = 1.708478656;
lumi_periods = {}
lumi_periods[ "2017B" ]  = L_B
lumi_periods[ "2017C1" ] = L_C1
lumi_periods[ "2017E" ]  = L_E
lumi_periods[ "2017F1" ] = L_F1
print ( lumi_periods )
lumi_total = np.sum( list( lumi_periods.values() ) )
print ( "Total luminosity = {}".format( lumi_total ) )

{'2017B': 2.360904801, '2017C1': 5.313012839, '2017E': 8.958810514, '2017F1': 1.708478656}
Total luminosity = 18.34120681


### Random experiments

In [9]:
if run_random_experiments:
    from random_experiment import *
    
    np.random.seed( 42 )

    # per period, arm
    systematics = {}
    #systematics[ "Xi" ] = ( systematics_Xi_X, systematics_Xi_Y )
    #systematics[ "Xi" ] = get_systematics_vs_xi_ROOT( data_periods, fileName="reco_characteristics/reco_characteristics_version1.root" )
    systematics[ "Xi" ] = get_systematics_vs_xi_h5( data_periods, fileName="reco_characteristics/reco_characteristics_version1.h5" )
   
    print ( systematics )
    
    random_experiment( df_signal, data_periods=data_periods, lumi_weights=lumi_periods, variables=[ "Xi" ], variations=systematics )
    

In [10]:
if run_random_experiments:
    print ( df_signal[ [ "Run", "LumiSection", "EventNum", "Arm", "Xi", "period_rnd", "sigma_Xi", "Xi_smeared"] ] )

### Background

In [11]:
resample_factor = 20
label = "data_random_resample_20"

fileNames_bkg = [
    'output/output-UL2017B-PreSel-Rnd-Res20.h5',
    'output/output-UL2017C1-PreSel-Rnd-Res20.h5',
    'output/output-UL2017E-PreSel-Rnd-Res20_0.h5',
    'output/output-UL2017E-PreSel-Rnd-Res20_1.h5',
    'output/output-UL2017F1-PreSel-Rnd-Res20.h5'
]

import time
print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
time_s_ = time.time()

df_counts_bkg, df_bkg = 2 * [None]

if run_tables:
    with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), complevel=5 ) as store_:

        df_counts_bkg_list_ = []
        df_bkg_list_ = []
        for file_ in fileNames_bkg:
            df_counts_bkg_, df_bkg_ = get_data( [ file_ ] )
            df_bkg_ = process_data( df_bkg_, proton_selection, min_mass = 110., within_aperture=True )
            df_counts_bkg_list_.append( df_counts_bkg_ )
            df_bkg_list_.append( df_bkg_ )

        df_counts_bkg_ = df_counts_bkg_list_[0]
        for idx_ in range( 1, len( df_counts_bkg_list_ ) ):
            df_counts_bkg_ = df_counts_bkg_.add( df_counts_bkg_list_[idx_] )

        df_bkg_ = pd.concat( df_bkg_list_ )
        
        store_[ "counts" ] = df_counts_bkg_
        store_[ "df" ] = df_bkg_

with pd.HDFStore( "reduced-data-store-{}.h5".format( label ), 'r' ) as store_:
    df_counts_bkg = store_[ "counts" ]
    df_bkg = store_[ "df" ]
        
time_e_ = time.time()
print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

2021/04/15 00:51:50
Total time elapsed: 1


In [12]:
df_counts_bkg

All           869119560
Muon            8456020
Vertex          7782360
ProtonsRnd      6301680
dtype: int64

In [13]:
df_bkg

,Run,LumiSection,EventNum,Slice,CrossingAngle,MultiRP,Arm,RPId1,RPId2,TrackX1,...,Acopl,XiMuMuPlus,XiMuMuMinus,XiMuMu,period,within_aperture,xlow,xhigh,ylow,yhigh
10,297227,56,38791640,0,120.0,1,0,3,23,7.063232,...,0.058782,0.002211,0.062855,0.002211,2017B,True,1.995,24.334,-10.098,4.298
20,297227,58,40362008,0,120.0,1,0,3,23,7.391113,...,0.271141,0.028539,0.011619,0.028539,2017B,True,1.995,24.334,-10.098,4.298
33,297227,60,41478176,0,120.0,1,0,3,23,6.782959,...,0.157789,0.035037,0.018197,0.035037,2017B,True,1.995,24.334,-10.098,4.298
47,297227,63,44189296,0,120.0,1,0,3,23,13.382812,...,0.008506,0.033806,0.010684,0.033806,2017B,True,1.995,24.334,-10.098,4.298
69,297227,73,52238048,0,120.0,1,1,103,123,8.629883,...,0.060892,0.014483,0.008333,0.008333,2017B,True,2.422,24.620,-9.698,4.698
84,297227,75,53449356,0,120.0,1,1,103,123,5.546875,...,0.023202,0.019344,0.007848,0.007848,2017B,True,2.422,24.620,-9.698,4.698
93,297227,76,54126336,0,120.0,1,0,3,23,8.200684,...,0.145185,0.049092,0.006129,0.049092,2017B,True,1.995,24.334,-10.098,4.298
111,297227,78,55898044,0,120.0,1,1,103,123,3.611084,...,0.273834,0.032531,0.003312,0.003312,2017B,True,2.422,24.620,-9.698,4.698
118,297227,79,56863080,0,120.0,1,1,103,123,3.396606,...,0.026694,0.001631,0.059633,0.059633,2017B,True,2.422,24.620,-9.698,4.698
138,297227,86,61943160,0,120.0,1,0,3,23,7.637451,...,0.095938,0.016999,0.009692,0.016999,2017B,True,1.995,24.334,-10.098,4.298


### Set aside test sample

In [14]:
from sklearn.model_selection import train_test_split

y_sig_ = np.ones( df_signal.shape[0] )
y_bkg_ = np.zeros( df_bkg.shape[0] )

df_signal_train, df_signal_test, y_sig_train, y_sig_test = train_test_split( df_signal, y_sig_, test_size=0.40, shuffle=True, random_state=12345 )
df_bkg_train, df_bkg_test, y_bkg_train, y_bkg_test = train_test_split( df_bkg, y_bkg_, test_size=0.40, shuffle=True, random_state=12345 )

print ( df_signal_train, df_signal_test, y_sig_train, y_sig_test )
print ( df_bkg_train, df_bkg_test, y_bkg_train, y_bkg_test )

        Run  LumiSection  EventNum  Slice  CrossingAngle  MultiRP  Arm  RPId1  \
7184      1          764     76394      0          120.0        1    0      3   
134812    1         1698    169732      0          150.0        1    1    103   
198101    1          414     41334      0          130.0        1    1    103   
45399     1         1124    112389      0          130.0        1    1    103   
1743      1           17      1693      0          150.0        1    0      3   
121218    1         1573    157270      0          120.0        1    1    103   
15170     1          841     84012      0          120.0        1    1    103   
37604     1         1049    104823      0          150.0        1    1    103   
210846    1          732     73107      0          150.0        1    0      3   
117203    1         1522    152180      0          120.0        1    1    103   
28055     1          961     96012      0          130.0        1    0      3   
86475     1         1251    

           Run  LumiSection    EventNum  Slice  CrossingAngle  MultiRP  Arm  \
584610  304169          775  1174086912     15          130.0        1    1   
305145  300375           71   110011792     10          140.0        1    0   
665428  297219          184   377924896      4          150.0        1    0   
988550  304209          366   622661056     12          140.0        1    1   
696031  297219          690  1276009216      6          130.0        1    0   
611262  304447         1056  1711461760      8          120.0        1    1   
106992  299481          305   363365184      0          120.0        1    0   
802514  304654          407   657480768      8          130.0        1    1   
391423  300576          119   174375040      8          140.0        1    1   
688733  305045          278   484990912     14          140.0        1    1   
886189  300280          635   846633792      5          140.0        1    0   
691408  305112          628  1070034240      8      

In [15]:
print ( [ arr_.shape[0] for arr_ in ( df_signal_train, df_signal_test, y_sig_train, y_sig_test ) ] )
print ( [ arr_.shape[0] for arr_ in ( df_bkg_train, df_bkg_test, y_bkg_train, y_bkg_test ) ] )
print ( df_signal_test.mean() )
# print ( df_bkg_test.mean() )

[28830, 19221, 28830, 19221]
[727796, 485198, 727796, 485198]
Run                       1.000000e+00
LumiSection               1.004417e+03
EventNum                  1.003923e+05
Slice                     0.000000e+00
CrossingAngle             1.335316e+02
MultiRP                   1.000000e+00
Arm                       4.798918e-01
RPId1                     5.098918e+01
RPId2                     7.098918e+01
TrackX1                   5.360522e+00
TrackY1                   8.681382e-01
TrackX2                   5.378507e+00
TrackY2                   9.039768e-01
TrackThX_SingleRP        -9.990000e+02
TrackThY_SingleRP        -9.990000e+02
Track1ThX_MultiRP        -6.571191e-06
Track1ThY_MultiRP         7.334872e-06
Track2ThX_MultiRP        -7.935825e-06
Track2ThY_MultiRP         6.452781e-06
TrackPixShift_SingleRP   -1.000000e+00
Track1PixShift_MultiRP    0.000000e+00
Track2PixShift_MultiRP    0.000000e+00
Xi                        5.838119e-02
T                        -1.802354e-01
Th

### Select variables

In [16]:
# X_sig = df_signal[ ['Xi', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ]
X_sig_train = None
X_sig_test = None
if run_random_experiments:
    X_sig_train = df_signal_train[ ['Xi_smeared', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ].rename( columns={ "Xi_smeared": "Xi" } )
    X_sig_test = df_signal_test[ ['Xi_smeared', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ].rename( columns={ "Xi_smeared": "Xi" } )
else:
    X_sig_train = df_signal_train[ ['Xi', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ]
    X_sig_test = df_signal_test[ ['Xi', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ]
X_sig_train

,Xi,Muon0Pt,Muon1Pt,InvMass,ExtraPfCands,Acopl,XiMuMu
7184,0.040329,73.456902,72.462585,178.602219,1,0.000010,0.040151
134812,0.039064,77.487175,76.224182,153.753983,2,0.000206,0.037886
198101,0.041670,63.803547,62.695671,166.183243,0,0.000691,0.043130
45399,0.058295,53.412079,52.910538,126.224342,1,0.000256,0.056360
1743,0.043040,53.542686,50.286655,125.001396,0,0.001747,0.041338
121218,0.040292,54.416470,54.017834,127.565163,0,0.002157,0.041076
15170,0.072571,67.009575,66.574005,138.386398,1,0.000247,0.070304
37604,0.065533,122.996933,120.216347,262.177338,0,0.000288,0.068566
210846,0.053142,67.908630,67.161713,135.474152,5,0.000149,0.050503
117203,0.055398,60.326309,59.703342,166.485199,2,0.000219,0.054283


In [17]:
X_bkg_train = df_bkg_train[ ['Xi', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ]
X_bkg_test = df_bkg_test[ ['Xi', 'Muon0Pt', 'Muon1Pt', 'InvMass', 'ExtraPfCands', 'Acopl', 'XiMuMu'] ]
X_bkg_train

,Xi,Muon0Pt,Muon1Pt,InvMass,ExtraPfCands,Acopl,XiMuMu
584610,0.057389,72.875313,55.133251,139.115662,18,0.107399,0.018423
305145,0.043718,110.621307,105.858086,239.736038,41,0.105640,0.051889
665428,0.096003,248.045914,129.451035,247.146591,13,0.552615,0.006062
988550,0.120712,154.062881,99.193192,416.161377,26,0.019761,0.016469
696031,0.111822,179.805893,176.773315,373.322815,11,0.017037,0.008327
611262,0.060924,77.290184,64.635056,148.078384,31,0.005014,0.003526
106992,0.032264,55.543747,53.829002,113.869637,31,0.030717,0.031280
802514,0.076775,86.059380,57.181171,157.845184,44,0.002613,0.002303
391423,0.052201,60.528744,55.078896,185.600433,22,0.000453,0.021551
688733,0.128419,58.373783,54.912369,113.674355,41,0.028492,0.004507


In [18]:
# y_sig = np.ones( len(X_sig) )
# y_bkg = np.zeros( len(X_bkg) )

# X = pd.concat( [X_sig, X_bkg] ) 
# y = np.concatenate( [y_sig, y_bkg] )

# from sklearn.model_selection import train_test_split
# X_train_full, X_test, y_train_full, y_test = train_test_split( X, y, test_size=0.20, shuffle=True, random_state=42 )
# X_train, X_valid, y_train, y_valid = train_test_split( X_train_full, y_train_full, test_size=0.20, shuffle=False )

In [19]:
X_ = pd.concat( [X_sig_train, X_bkg_train] ) 
y_ = np.concatenate( [y_sig_train, y_bkg_train] )

X_train, X_valid, y_train, y_valid = train_test_split( X_, y_, test_size=0.20, shuffle=True, random_state=42 )

In [20]:
X_train

,Xi,Muon0Pt,Muon1Pt,InvMass,ExtraPfCands,Acopl,XiMuMu
469472,0.058105,72.826210,55.110813,130.711975,25,0.040807,0.040254
609700,0.091873,53.075546,52.507694,118.192848,37,0.085519,0.005388
682659,0.075423,231.828979,220.913254,460.083496,49,0.015712,0.147346
77778,0.036630,163.956985,151.021820,315.850922,5,0.006405,0.009379
347724,0.044258,135.377182,78.774803,196.752762,78,0.214974,0.014618
30275,0.162471,67.015663,56.636086,160.171722,20,0.030221,0.033495
327758,0.054789,106.260559,79.086670,216.952545,21,0.161898,0.006411
624191,0.099288,59.962311,51.655449,122.957054,41,0.011462,0.018061
660197,0.061193,61.142776,55.399841,171.535370,35,0.047100,0.037515
485678,0.038942,59.169373,52.536217,110.931953,32,0.064774,0.001647


In [21]:
X_test = pd.concat( [X_sig_test, X_bkg_test] ) 
y_test = np.concatenate( [y_sig_test, y_bkg_test] )

In [22]:
X_test

,Xi,Muon0Pt,Muon1Pt,InvMass,ExtraPfCands,Acopl,XiMuMu
99234,0.076666,299.772003,293.133179,783.843018,3,8.626284e-07,0.077529
140279,0.041093,141.848022,139.301285,400.948456,12,5.804421e-04,0.042499
190165,0.055388,84.725891,83.376411,223.688828,3,2.251787e-04,0.056514
148266,0.051295,96.573059,96.415550,218.593445,0,7.299701e-05,0.049608
44734,0.048346,52.776470,52.612209,111.511894,5,6.226767e-04,0.047299
205320,0.098880,117.329910,113.676643,236.842682,6,2.636554e-04,0.100070
111452,0.052445,52.369503,52.356544,120.156525,1,1.043757e-03,0.052469
84328,0.043292,52.813648,52.537960,151.280701,0,1.070642e-03,0.041663
191766,0.075327,57.417480,53.987713,116.916412,0,7.648572e-04,0.070977
7014,0.073473,78.993111,73.476295,155.143829,0,3.987060e-05,0.074484


In [ ]:
np.sum( y_train == 1 ) / y_train.size

In [ ]:
np.sum( y_valid == 1 ) / y_valid.size

In [ ]:
np.sum( y_test == 1 ) / y_test.size

### Scale inputs

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = None
X_train_scaled = None
if train_model:
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform( X_train )
else:
    scaler = load( scaler_path )
    X_train_scaled = scaler.transform( X_train )
    
X_valid_scaled = scaler.transform( X_valid )
X_test_scaled = scaler.transform( X_test )

print ( scaler )

if train_model and save_model:
    dump( scaler, "model/standard_scaler.joblib")

In [ ]:
X_train_scaled[:20]

In [ ]:
nvars = X_train_scaled.shape[1]
ncols = 3
nrows = ( nvars // ncols ) if nvars % ncols == 0 else ( nvars // ncols + 1 )

fig, axes = plt.subplots( nrows, ncols, figsize=( ncols * 12, nrows * 10 ) )

idx_row = 0
idx_col = 0
for idx_ in range( nvars ):
    arr_ = X_train_scaled[ :, idx_ ]
    axes[ idx_row, idx_col ].hist( arr_, bins=50 )
    idx_col += 1
    if idx_col == ncols:
        idx_row += 1
        idx_col = 0

### Model build function

In [ ]:
# def build_model(n_hidden=1, n_neurons=50, learning_rate=5e-4, dropout=0.20, input_shape=X_train_scaled.shape[1:] ):
#     print( "Building model with:" )
#     print( "Number of hidden layers: {}".format(n_hidden) )
#     print( "Number of neurons per layer: {}".format(n_neurons) )
#     print( "Learning rate: {}".format(learning_rate) )
#     print( "Input shape: {}".format(input_shape) )
#     print( "Dropout rate: {}".format(dropout) )
    
#     model = keras.models.Sequential()
#     model.add( keras.layers.InputLayer(input_shape=input_shape) )
#     for layer in range(n_hidden):
#         if dropout > 0.:
#             model.add( keras.layers.Dropout(rate=dropout) )
#         model.add( keras.layers.Dense(n_neurons, activation="elu", kernel_initializer="he_normal") )
#     if dropout > 0.:
#         model.add( keras.layers.Dropout(rate=dropout) )    
#     model.add( keras.layers.Dense(1, activation="sigmoid") )
    
#     #optimizer = keras.optimizers.SGD(lr=learning_rate, momentum=0.9, nesterov=True)
#     optimizer = keras.optimizers.Nadam(lr=learning_rate)
#     model.compile( loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    
#     return model

In [ ]:
from keras_model import Model, build_model

### Define training callbacks

In [ ]:
def get_run_logdir(log_dir):
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(log_dir, run_id)

In [ ]:
def callbacks(patience=10, log_dir=""):
    callbacks_ = []
    # Early stopping
    if patience > 0:
        early_stopping_cb_ = keras.callbacks.EarlyStopping( patience=patience, restore_best_weights=True )
        callbacks_.append( early_stopping_cb_ )
        
    # TensorBoard
    if log_dir:
        run_logdir = get_run_logdir(log_dir)
        print ( "Log dir: {}".format(run_logdir) )
        tensorboard_cb_ = keras.callbacks.TensorBoard( run_logdir )
        callbacks_.append( tensorboard_cb_ )
    
    return callbacks_

In [ ]:
log_dir="keras_logs"
callbacks_ = callbacks(patience=5, log_dir=log_dir)
callbacks_

### Build model (example)

In [ ]:
model = build_model( 
    input_shape=X_train_scaled.shape[1:],
    learning_rate=1e-4,
    n_hidden=1,
    n_neurons=50,
    dropout=0.20
    )
model.summary()

In [ ]:
fit_example = False

if fit_example:
    history = model.fit( X_train_scaled, y_train, epochs=20, batch_size=32, validation_data=(X_valid_scaled, y_valid), callbacks=callbacks_ )
    print( history.history )

    pd.DataFrame( history.history ).plot( figsize=(12,10) )

### Evaluate on training data (without dropout)

In [ ]:
if fit_example:
    model.evaluate( X_train_scaled, y_train )

### Evaluate on test data

In [ ]:
if fit_example:
    model.evaluate( X_test_scaled, y_test )

    y_test_proba = model.predict( X_test_scaled )
    print ( y_test_proba.shape )
    print ( y_test_proba )

### Scan over different learning rates

In [ ]:
def find_max_learning_rate( lr_init=1e-4, lr_end=5e-2, steps=20, epochs=20, batch_size=32, model_build_fn=build_model, *build_fn_args, **build_fn_kwargs ):
    results_ = {}
    results_['learning_rate'] = []
    results_['loss'] = []
    results_['accuracy'] = []
    results_['val_loss'] = []
    results_['val_accuracy'] = []
    c_ = (lr_end/lr_init) ** (1/steps)
    lr_ = lr_init
    for i_it in range( steps + 1 ):
        results_['learning_rate'].append( lr_ )
        model_ = model_build_fn( *build_fn_args, **build_fn_kwargs, learning_rate=lr_ )
        callbacks_ = callbacks(patience=5)
        history_ = model_.fit( X_train_scaled, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid_scaled, y_valid), callbacks=callbacks_ )
        results_['loss'].append( history_.history['loss'] )
        results_['accuracy'].append( history_.history['accuracy'] )
        results_['val_loss'].append( history_.history['val_loss'] )
        results_['val_accuracy'].append( history_.history['val_accuracy'] )
        # Update learning rate
        lr_ = lr_ * c_
        
    return results_

In [ ]:
epochs_lr_scan=20
batch_size=1024
results = None
if learning_rate_scan:
    results = find_max_learning_rate(
                lr_init=1e-4,
                lr_end=2e-2,
                steps=10,
                epochs=epochs_lr_scan,
                batch_size=batch_size,
                model_build_fn=build_model,
                input_shape=X_train_scaled.shape[1:],
                n_hidden=1,
                n_neurons=20,
                dropout=0.20
                )

In [ ]:
if learning_rate_scan:
    metrics_ = 'val_loss'
    columns=["lr_{}".format(lr_) for lr_ in np.round( results['learning_rate'], 4)] 
    df = pd.DataFrame( np.full((epochs_lr_scan,len(columns)),np.nan), columns=columns )
    for i_lr_,col_ in enumerate(columns):
        df[col_] = pd.Series( results[ metrics_ ][i_lr_] )
    df.plot( figsize=(12,10) )
    plt.yscale('log')
    plt.show()

### Hyperparameter scan

In [ ]:
learning_rate = 5e-4

n_iter_search = 3
epochs_grid_search = 20

grid_search = None
if train_model and run_grid_search:
    import time
    print( time.strftime("%Y/%m/%d %H:%M:%S", time.localtime() ) )
    time_s_ = time.time()

    from sklearn.model_selection import RandomizedSearchCV
    #from sklearn.model_selection import GridSearchCV

    build_fn_ = Model( input_shape=X_train_scaled.shape[1:], learning_rate=learning_rate )
    keras_clf = keras.wrappers.scikit_learn.KerasClassifier( build_fn_ )

#     #param_grid = [
#     #    { "n_hidden": [2],
#     #      "n_neurons": [50,100] }
#     #    ]
#     param_grid = [
#         { "n_hidden": np.arange(1,3),
#           "n_neurons": [20,50] }
#         ]

    param_distribs = {
        "n_hidden": np.arange(2,6),
        "n_neurons": 2 ** np.arange(4,8),
        "dropout":  0.1 * np.arange(2,6),
        "batch_size": 2 ** np.arange(5,8)
        }

    #grid_search = GridSearchCV( keras_clf, param_grid, cv=3, scoring='f1', refit=False )
    
    grid_search = RandomizedSearchCV(
        keras_clf,
        param_distribs,
        n_iter=n_iter_search, cv=4, verbose=20, n_jobs=-1, scoring='f1', refit=False, random_state=42
        )

    callbacks_ = callbacks(patience=5)
    print ( callbacks_ )
    grid_search.fit( X_train_scaled, y_train, epochs=epochs_grid_search, validation_data=(X_valid_scaled, y_valid), callbacks=callbacks_ )
    
    print ( grid_search.best_params_ )
    print ( grid_search.best_score_ )
    print ( grid_search.cv_results_ )
    
    time_e_ = time.time()
    print ( "Total time elapsed: {:.0f}".format( time_e_ - time_s_ ) )

### Build model

In [ ]:
model_final = None

if train_model:
    params = {'n_hidden': 1, 'n_neurons': 50, 'dropout': 0.20}
    batch_size = 32
    if run_grid_search: 
        params = grid_search.best_params_.copy()
        batch_size = params[ 'batch_size' ]
        params.pop( 'batch_size' )
    print ( params, "batch_size: {}".format( batch_size ) )
    
    model_final = build_model(input_shape=X_train_scaled.shape[1:], learning_rate=learning_rate, **params)
    model_final.summary()
    log_dir="keras_logs"
    callbacks_ = callbacks(patience=5, log_dir=log_dir)
    print ( callbacks_ )
    model_final.fit( X_train_scaled, y_train, epochs=100, batch_size=batch_size, validation_data=(X_valid_scaled, y_valid), callbacks=callbacks_ )
else:
    model_final = keras.models.load_model( model_path )
    
model_final.summary()

### Evaluate on training data (without dropout)

In [ ]:
model_final.evaluate( X_train_scaled, y_train )

### Re-evaluate on validation data 

In [ ]:
model_final.evaluate( X_valid_scaled, y_valid )

### Evaluate on test data

In [ ]:
model_final.evaluate( X_test_scaled, y_test )

y_test_proba = model_final.predict( X_test_scaled )
y_test_proba

In [ ]:
fig = plt.figure( figsize=(12,10) )
plt.hist( y_test_proba[ y_test == 0 ], histtype='step', color='orange', bins=60, range=(0.,1.) )
plt.hist( y_test_proba[ y_test == 1 ], histtype='step', color='skyblue', bins=60, range=(0.,1.) )
plt.yscale('log')

In [ ]:
prob_cut = 0.20

y_test_pred = ( y_test_proba >= prob_cut ).astype( "int32" )
print ( y_test_pred )

from sklearn.metrics import accuracy_score
print ( accuracy_score( y_test, y_test_pred ) )
print ( accuracy_score( y_test[ y_test == 1 ], y_test_pred[ y_test == 1 ] ) )
print ( accuracy_score( y_test[ y_test == 0 ], y_test_pred[ y_test == 0 ] ) )

### Save model

In [ ]:
if train_model and save_model:
    model_final.save( "model/keras_model.h5" )

### References

In [ ]:
from sklearn.model_selection import GridSearchCV
np.info(GridSearchCV)

In [ ]:
from sklearn.model_selection import train_test_split
np.info(train_test_split)

In [ ]:
help(model)